# Tugas 3.2 : Uji Coba Model Logistic Regression

Pada Tugas 3.2 ini diminta untuk melakukan uji coba model yang telah dibuat dengan algoritma Logistic Regression.

Dibuat Oleh:

*   Nama : Sabil Ahmad Hidayat
*   NIM : 220411100058
*   Kelas : PPW A

Link Code : https://colab.research.google.com/drive/1UBJt_jNWAnwp-Jhg_McgcuYY6zahVKLW?usp=sharing

Link Github : https://github.com/meinhere/ppw/tree/master/publish/tugas-3

## Import Library

In [ ]:
!pip install -q Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd

# alat untuk crawling
from urllib.request import urlopen
from bs4 import BeautifulSoup

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# library untuk proses modeling
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# library untuk evaluasi model
from sklearn.metrics import classification_report, confusion_matrix

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# save model
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**preprocessing** disini digunakan untuk melakukan proses encoding pada label

**train_test_split** digunakan untuk membagi dataset menjadi data training dan testing

**LogisticRegression** digunakan untuk tahap modeling menggunakan library LogisticRegression

**classification_report** dan **confusion_matrix** digunakan untuk melihat laporan dan hasil evaluasi setelah proses training data

**matplotlib** dan **seaborn** digunakan untuk plotting grafik

**pickle** digunakan untuk menyimpan model hasil training dan testing

## Persiapan Data

### Load Data Model

In [ ]:
main_df = pd.read_csv('https://raw.githubusercontent.com/meinhere/ppw/master/publish/tugas-2/data_berita.csv', delimiter=',')
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,Simak Jadwal dan Lokasi SIM Keliling di Jakart...,"JAKARTA, KOMPAS.com - Surat Izin Mengemudi (S...",07/09/2024,OTOMOTIF
1,2,[POPULER OTOMOTIF] Diskon Motor Honda Septembe...,"JAKARTA, KOMPAS.com - Banyak pembaca yang ingi...",07/09/2024,OTOMOTIF
2,3,"Cek Saldo Minimal BRI, BNI, BCA, Mandiri, dan BSI","JAKARTA, KOMPAS.com - Penting bagi calon nasab...",06/09/2024,MONEY
3,4,"KAI Uji Coba Teknologi ""Face Recognition Board...",KOMPAS.com - PT Kereta Api Indonesia (KAI) Div...,06/09/2024,MONEY
4,5,OJK Blokir 10.890 Entitas Keuangan Ilegal Seja...,"JAKARTA, KOMPAS.com - Otoritas Jasa Keuangan (...",06/09/2024,MONEY
...,...,...,...,...,...
95,96,Waspada Masalah yang Timbul akibat Telat Ganti...,"JAKARTA, KOMPAS.com - Oli mesin pada mobil den...",06/09/2024,OTOMOTIF
96,97,"Sosok Faisal Basri di Mata Para Tokoh, Ekonom ...","JAKARTA, KOMPAS.com - Ekonom senior Faisal Bas...",06/09/2024,MONEY
97,98,"Pendaftaran CPNS Diperpanjang 4 Hari, Pelamar ...","JAKARTA, KOMPAS.com - Pemerintah telah memperp...",06/09/2024,MONEY
98,99,"Harga Emas Terbaru Pegadaian, Jumat 6 Septembe...","JAKARTA, KOMPAS.com - Pegadaian menyediakan be...",06/09/2024,MONEY


### Membuat Fungsi untuk Persiapan Crawling

In [ ]:
# fungsi untuk mengambil link yang akan dilakukan crawling
def extract_urls(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    urls = soup.find_all("a", {"class": "paging__link"})
    urls = [url.get('href') for url in urls]

    return urls

# fungsi untuk mengambil isi dari berita
def get_content(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    div = soup.find("div", {"class": "read__content"})
    paragraf = div.find_all("p")

    content = ''
    for p in paragraf:
        content += p.text

    return content


# fungsi utama crawling
def crawl(link = "https://indeks.kompas.com", max_money = 1, max_otomotif = 1, allow_category = ["OTOMOTIF", "MONEY"], is_train = True, title_old = []):
    # inisialisasi variabel penampung hasil berita
    news_data = []

    # inisialisasi persiapan untuk crawling berita
    last_url = extract_urls(link).pop()
    page = last_url.split('=').pop() # jumlah halaman secara otomatis
    # page = 1 # jumlah halaman secara manual

    # persiapan link yang akan dilakukan crawling
    urls = [link + '/?page=' + str(a) for a in range(1, int(page) + 1)]
    count_money = 0
    count_otomotif = 0

    # menelusuri semua link yang telah ditentukan
    for idx, url in enumerate(urls):
        if (len(news_data) == max_money + max_otomotif) :
          break

        html = urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        # mengambil data yang diperlukan pada struktur html
        links       = soup.find_all("a", {"class": "article-link"})
        titles      = soup.find_all("h2", {"class": "articleTitle"})
        dates       = soup.find_all("div", {"class": "articlePost-date"})
        categories  = soup.find_all("div", {"class": "articlePost-subtitle"})

        news_per_page = len(links) # berita artikel yang ditampilkan

        # memasukkan data ke dalam list
        for elem in tqdm(range(news_per_page), desc=f"Crawling page {idx+1}"):
          news = {}
          category = categories[elem].text
          title = titles[elem].text

          if (category in allow_category):
            if (is_train):
              cond = (category == "MONEY" and count_money < max_money) or (category == "OTOMOTIF" and count_otomotif < max_otomotif)
            else:
              cond = (category == "MONEY" and count_money < max_money) or (category == "OTOMOTIF" and count_otomotif < max_otomotif) and title not in title_old


            if (cond):
              news['No'] = len(news_data) + 1
              news['Judul Berita']     = title
              news['Isi Berita']       = get_content(links[elem].get("href"))
              news['Tanggal Berita']   = dates[elem].text
              news['Kategori Berita']  = category
              news_data.append(news)

              if (category == "MONEY"):
                count_money += 1
              else:
                count_otomotif += 1

        print(f"=======> Money: {count_money} | Otomotif: {count_otomotif} | Total: {count_money + count_otomotif}")

    return news_data

function **extract_urls** digunakan untuk melakukan ekstraksi link url yang memiliki pagination pada halaman awal, sehingga didapat beberapa url yang bisa mengarah ke halaman selanjutnya atau sebelumnya.

function **get_content** digunakan untuk melakukan proses pembuatan isi berita sesuai link berita yang dicari.

### Pengambilan Data Baru

In [ ]:
title_old = main_df["Judul Berita"].tolist()

test_news = crawl(max_money=5, max_otomotif=5, is_train=False, title_old=title_old)

Crawling page 1: 100%|██████████| 15/15 [00:06<00:00,  2.50it/s]


=======> Money: 3 | Otomotif: 0 | Total: 3


Crawling page 2: 100%|██████████| 15/15 [00:03<00:00,  4.84it/s]


=======> Money: 5 | Otomotif: 0 | Total: 5


Crawling page 3: 100%|██████████| 15/15 [00:02<00:00,  5.73it/s]


...

=======> Money: 5 | Otomotif: 3 | Total: 8


Crawling page 6: 100%|██████████| 15/15 [00:01<00:00, 10.13it/s]


=======> Money: 5 | Otomotif: 4 | Total: 9


Crawling page 7: 100%|██████████| 15/15 [00:01<00:00,  9.23it/s]

=======> Money: 5 | Otomotif: 5 | Total: 10


In [ ]:
main_df = pd.DataFrame(test_news)
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,Dana Kelolaan Reksa Dana BRI Gamasteps Tembus ...,"JAKARTA, KOMPAS.com - Reksa dana pasar uang BR...",16/10/2024,MONEY
1,2,"Pelanggaran Aturan Wajib SNI, Kemenperin: Sank...","JAKARTA, KOMPAS.com - Kepala Pusat Pengawasan ...",16/10/2024,MONEY
2,3,Mark Dynamics Catat Penjualan Naik 173 Persen ...,"JAKARTA, KOMPAS.com - Emiten produsen sarung t...",16/10/2024,MONEY
3,4,"Produksi Beras 2024 Diprediksi Turun, Ini Peny...","JAKARTA, KOMPAS.com - Kementerian Pertanian (K...",16/10/2024,MONEY
4,5,"Per Hari Ini, KA Jayabaya Pakai Rangkaian Stai...",KOMPAS.com - Kereta api (KA) Jayabaya rute Mal...,16/10/2024,MONEY
5,6,Hitung Biaya Servis Honda Stylo 160 CBS,"JAKARTA, KOMPAS.com – Honda Stylo 160 CBS seca...",16/10/2024,OTOMOTIF
6,7,"Tidak Boleh Sembarangan, Begini Prosedur Razia...","JAKARTA, KOMPAS.com – Razia lalu lintas merupa...",16/10/2024,OTOMOTIF
7,8,"Hitung Konsumsi BBM Honda Stylo 160 CBS, Tembu...","JAKARTA, KOMPAS.com – Honda Stylo 160 CBS meru...",16/10/2024,OTOMOTIF
8,9,"Spesifikasi Lengkap Yamaha Grand Filano, Mesin...","JAKARTA, KOMPAS.com - Yamaha Grand Filano jadi...",16/10/2024,OTOMOTIF
9,10,Mobil Presiden Mercedes-Benz S600 Guard dan S6...,"JAKARTA, KOMPAS.com - Mobil dinas Presiden RI ...",16/10/2024,OTOMOTIF


## Praproses Teks

### Membuat Fungsi

In [ ]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\[\]\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords
def clean_stopwords(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

function **clean_lower** digunakan untuk merubah semua kata atau huruf menjadi huruf kecil semua

function **clean_punct** digunakan untuk menghapus karakter, simbol, dan angka

function **_normalize_whitespace** digunakan untuk menghapus spasi yang double atau lebih dari 2 spasi

function **clean_stopwords** digunakan untuk menghilangkan kata yang tidak perlu (kata hubung, kata tambahan dll)

function **sastrawistemmer** digunakan untuk proses stemming (mendapatkan kata dasar dari suatu kata)

### Clean Lower

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses case folding
main_df['lwr'] = main_df['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df['lwr'])
casefolding

,lwr
0,"jakarta, kompas.com - reksa dana pasar uang br..."
1,"jakarta, kompas.com - kepala pusat pengawasan ..."
2,"jakarta, kompas.com - emiten produsen sarung t..."
3,"jakarta, kompas.com - kementerian pertanian (k..."
4,kompas.com - kereta api (ka) jayabaya rute mal...
5,"jakarta, kompas.com – honda stylo 160 cbs seca..."
6,"jakarta, kompas.com – razia lalu lintas merupa..."
7,"jakarta, kompas.com – honda stylo 160 cbs meru..."
8,"jakarta, kompas.com - yamaha grand filano jadi..."
9,"jakarta, kompas.com - mobil dinas presiden ri ..."


### Clean Punct

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan tanda baca
main_df['clean_punct'] = main_df['lwr'].apply(clean_punct)
main_df['clean_punct']

,clean_punct
0,jakarta kompas com reksa dana pasar uang bri...
1,jakarta kompas com kepala pusat pengawasan s...
2,jakarta kompas com emiten produsen sarung ta...
3,jakarta kompas com kementerian pertanian kem...
4,kompas com kereta api ka jayabaya rute malan...
5,jakarta kompas com honda stylo cbs secara h...
6,jakarta kompas com razia lalu lintas merupak...
7,jakarta kompas com honda stylo cbs merupaka...
8,jakarta kompas com yamaha grand filano jadi ...
9,jakarta kompas com mobil dinas presiden ri j...


### Normalize Whitespace

In [ ]:
main_df['clean_double_ws'] = main_df['clean_punct'].apply(_normalize_whitespace)
main_df['clean_double_ws']

,clean_double_ws
0,jakarta kompas com reksa dana pasar uang bri g...
1,jakarta kompas com kepala pusat pengawasan sta...
2,jakarta kompas com emiten produsen sarung tang...
3,jakarta kompas com kementerian pertanian kemen...
4,kompas com kereta api ka jayabaya rute malang ...
5,jakarta kompas com honda stylo cbs secara harg...
6,jakarta kompas com razia lalu lintas merupakan...
7,jakarta kompas com honda stylo cbs merupakan s...
8,jakarta kompas com yamaha grand filano jadi pi...
9,jakarta kompas com mobil dinas presiden ri jok...


### Clean Stopwords

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df['clean_sw'] = main_df['clean_double_ws'].apply(clean_stopwords)
main_df['clean_sw']

,clean_sw
0,jakarta kompas com reksa dana pasar uang bri g...
1,jakarta kompas com kepala pusat pengawasan sta...
2,jakarta kompas com emiten produsen sarung tang...
3,jakarta kompas com kementerian pertanian kemen...
4,kompas com kereta api ka jayabaya rute malang ...
5,jakarta kompas com honda stylo cbs harga dijua...
6,jakarta kompas com razia lintas kegiatan penga...
7,jakarta kompas com honda stylo cbs skutik berg...
8,jakarta kompas com yamaha grand filano pilihan...
9,jakarta kompas com mobil dinas presiden ri jok...


### Stemming dengan Sastrawi

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilemmatization
main_df['desc_clean_stem'] = main_df['clean_sw'].apply(sastrawistemmer)
main_df['desc_clean_stem']

100%|██████████| 220/220 [00:08<00:00, 26.96it/s]


,desc_clean_stem
0,jakarta kompas com reksa dana pasar uang bri g...
1,jakarta kompas com kepala pusat awas standar i...
2,jakarta kompas com emiten produsen sarung tang...
3,jakarta kompas com menteri tani tan tanggap ba...
4,kompas com kereta api ka jayabaya rute malang ...
5,jakarta kompas com honda stylo cbs harga jual ...
6,jakarta kompas com razia lintas giat awas apar...
7,jakarta kompas com honda stylo cbs skutik gaya...
8,jakarta kompas com yamaha grand filano pilih s...
9,jakarta kompas com mobil dinas presiden ri jok...


## Pembuatan VSM

In [ ]:
# Load the saved model from file
filename = 'tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename, 'rb'))

In [ ]:
corpus = main_df['desc_clean_stem']
tfidf = tfidf_vectorizer.transform(corpus)

tfidf.shape

(10, 3555)

In [ ]:
vocabulary = tfidf_vectorizer.get_feature_names_out().tolist()

tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vocabulary)
tfidf_df.insert(0, 'Kategori Berita', main_df['Kategori Berita'])
tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,MONEY,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MONEY,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MONEY,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MONEY,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MONEY,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,OTOMOTIF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,OTOMOTIF,0.0,0.0,0.0,0.056221,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,OTOMOTIF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,OTOMOTIF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,OTOMOTIF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preparing Data Test

### Encode Label
Dilakukan tahap encoding pada kolom **[Kategori Berita]** dimana data yang yang terdapat didalamnya masih berupa data kategorik (kata) sehingga perlu dirubah menjadi angka agar bisa dimasukkan ke dalam proses training model

In [ ]:
# menggunakan label_encoder untuk merubah kata menjadi angka
label_encoder = preprocessing.LabelEncoder()
tfidf_df['Kategori Berita'] = label_encoder.fit_transform(tfidf_df['Kategori Berita'])

tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,0.0,0.0,0.0,0.056221,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Testing Data

In [ ]:
# Load the saved model from file
filename = 'lr_model.sav'
lr_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_test = tfidf_df['Kategori Berita']
x_test = tfidf_df.drop(['Kategori Berita'], axis=1)
y_pred = lr_model.predict(x_test)

print(y_pred)

[0 0 0 0 0 1 1 1 1 1]


In [ ]:
# melihat nilai actual dan predicted
a = pd.DataFrame({'Actual value': y_test, 'Predicted value':y_pred})
a

,Actual value,Predicted value
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1
